In [13]:
#初始條件
L = 1.0
K,T,Tb,Tc,p0=8.314,273.15,373.15,173.15,101300
theta_rod=pi
theta_diff=pi/2
omega=-2*pi
t0=0
t,dt=0,1e-3

#display
scene=display(width=800,height=800,x=200,y=0,background=(0.2,0.2,0),center=(2*L,0,0))
#container
container=cylinder(pos=(0,0,0),axis=(3*L,0,0),radius=L/2,opacity=0.2)
#轉盤
wheel_pos=vector(5*L,0,0)
wheel=cylinder(pos=wheel_pos,axis=(0,0,L/10),radius=L/2,material=materials.wood,opacity=0.3)

rod_lengh=L
rodco_lengh=0.5*L
rodho_lengh=3*L
rodc_x=L*cos(theta_rod)/2
rodc_y=L*sin(theta_rod)/2
rodc=cylinder(pos=wheel.pos+(rodc_x,rodc_y,0),axis=(-(L**2-rodc_y**2)**0.5,-rodc_y,0),radius=L/30,color=(0,0,0.3))
rodco=cylinder(pos=rodc.pos*rodc.axis,axis=(0,0,0),radius=L/30,color=(0,0.5,0.5),opcity=0.3)
rodco.axis=vector(-(rodco_lengh**2-rodco.pos.y**2)**0.5,-rodc.pos.y,0)

rodh_x=L*cos(theta_rod-theta_diff)/2
rodh_y=L*sin(theta_rod-theta_diff)/2
rodh=cylinder(pos=wheel.pos+(rodh_x,rodh_y,0),axis=((-L**2-rodh_y**2)**0.5,-rodh_y,0),radius=L/50,color=(0.3,0,0))
rodh=cylinder(pos=rodh.pos+rodh.axis,axis(0,0,0),radius=L/50,color=color(0.5,0.5,0))
rodh.axis=(-(rodho_lengh**2-rodho.pos.y**2)**0.5,-rodho.pos.y,0)

ratio=1
diskh=cylinder(pos=rodho.pos*rodho.axis,axis=(0.01*L,0,0),radius=ratio*L/2)
diskc=cylinder(pos=rodco.pos*rodco.axis,axis=(0.01*L,0,0),radius=L/2)

#函式庫
Vsh=pi*L**3/4
Vdh=pi*L**2*(wheel_pos.x-wheel.radius-rod_lengh-rodho_lengh)/4
Vsc=Vsh
Vdc=pi*L**2*(rodho_lengh-rodco_lengh-2*wheel.radius)/4

n=p0*pi*(wheel_pos.x-rod_lengh-rodco_lengh)*L**2/4/R/T
t_T=Tc/Th

def Vh():#熱庫體積
    return pi*L**2*(diskh.pos.x)/4
def Vc():#冷庫體積
    return pi*L**2*(diskc.pos.x)/4-pi*L**2*(diskh.pos.x)/4
def V(): #總體積
    return pi*L**2*(diskc.pos.x)/4
def p(): #壓力
    return (n*R*Tc/(t_T*Vh()+Vc()))

#熱力學參數
Qin=0
Qout=0
v=0
P=p()
v=V()
vh=Vh()
vc=Vc()

#圖形輸出
scene1=gdisplay(y=400,width=800,height=800,xtitle='V(m**3)',ytitle='P(Pa)',background=(0.5,0.5,0))
pV=gcurve(color=color.red,gdisplay=scene1)
scene2=gdisplay(y=400,width=800,height=800,xtitle='t(s)',ytitle='W(J),Qin(J)',background=(0.4,0.4,0))
scene3=gdisplay(y=400,width=800,height=800,xtitle='t(s)',ytitle='thermal efficiency',background=(0.4,0.4,0))
Wdrav=gcurve(color=color.cyan,gdisplay=scene2)#W
Qindrav=gcurve(color=color.red,gdisplay=scene2)#Qin
efficiency_drav=gdots(color=color.blue,gdisplay=scene3)#thermal efficiency
rfefdrav=gdots(color=color.cyan,gdisplay=scene3)#theory_thermal efficiency

while True:
    t+=dt
    rate(0.1/dt)
    
    #處理轉動
    theta_original=theta_rod
    theta_rod+=omega*dt
    wheel.rotate(angle=theta_rod-theta_original)
    
    #操作桿部分
    rodc_x=L*cos(theta_rod)/2
    rodc_y=L*sin(theta_rod)/2
    rodc.pos=vector(rodc_x,rodx_y,0)+wheel.pos
    rodc.axis=vector(-(L**2-rodc_y**2)**0.5,-rodc_y,0)
    rodco.pos=rodc.pos+rodc.axis
    rodco.axis=vector(-(rodco_lengh**2-rodco.pos.y**2)**0.5,-rodco.pos.y,0)
    
    rodh_x=L*cos(theta_rod-theta_diff)/2
    rodh_y=L*sin(theta_rod-theta_diff)/2
    rodh.pos=vector(rodh_x,rodh_y,0)+wheel.pos
    rodh.axis=vector(-(L**2-rodh_y**2)**0.5,-rodh_y,0)
    rodho.pos=rodh.pos+rodh.axis
    rodho.axis=vector(-(rodho_lengh**2-rodho.pos.y**2)**0.5,-rodho.pos.y,0)
    
    #活塞部分
    diskc.pos=rodco.pos+rodco.axis
    diskh.pos=rodho.pos+rodho.axis
    
    #容器部分
    containerh.axis=diskh.pos-vector(0,0,0)
    containerc.pos=diskh.pos
    containerc.axis=diskc.pos-diskh.pos
    container.pos=diskc.pos
    container.axis=vector(3*L,0,0)-diskc.pos
    
    #熱力過程
    W+=1.5*(P+p())*(V()-v)
    Qin+=1.5*(P+p())*(Vh()-vh)
    if(theta_rod-theta_rod0)+(theta_rod-theta_rod0+omega*dt)<=0: #每週期輸出一次
        Wdrav.plot(pos=(t,W))
        Qindrav.plot(pos=(t,Qin))
        if 0<W/Qin<1:
            efficiency_drav.plot(pos=(t,W/Qin))
            
        theta_rod0=-2*pi
    P=p()
    v=V()
    vh=Vh()
    vc=Vc()
    pV.plot(pos=(V(),p()))
    
    

SyntaxError: positional argument follows keyword argument (<ipython-input-13-050ab935b69b>, line 30)

In [11]:
import math
#Variable Declaration: 
n = 0.5 #Efficiency of carnot cycle:
m = 0.5 #Mass of air(in kg):
p2 = 7*10**5 #Initial pressure(in Pa):
v2 = 0.12 #Initial volume(in m**3):
Q23 = 40 #Heat transferred during the process 2-3(in kJ):
Cp = 1.008 #Specific heat at const pressure(in kJ/kg):
Cv = 0.721 #Specific heat at const volume(in kJ/kg):
Ra = 287 #Gas constant for air:
Q12 = 0 #Heat transfer in process 1-2(in kJ):
Q34 = 0 #Heat transfer in process 3-4(in kJ):

#Calculation:
T2 = p2*v2/(m*Ra)   #Maximum temperature of the cycle(in K):
T1 = T2/2  #Minimum temperature(in K):
v3 = v2*(math.e**(Q23/(m*Ra*10**(-3)*T2)))  #Volume at state 3(in m**3):
r = Cp/Cv  #Compression factor:
p1 = p2/((T2/T1)**(r/(r-1)))   #Pressure at point 1(in Pa):
v1 = m*Ra*T1/p1    #Volume at point 1(in m**3):
T3 = T2        #Temperature at state 3(in K):
T4 = T1   #Temperature at state 4(in K):
W12 = -m*Cv*(T2-T1)  #During process 1-2, work done(in kJ):
W23 = Q23 #Work done in process 2-3(in kJ):
W34 = -m*Cv*(T4-T3)  #During process 3-4, work done(in kJ):
W41 = -W23  #During process 4-1, work done(in kJ):
Q41 = -Q23 #Heat transfer in process 4-1(in kJ):

#Results:
print("Process                    Heat transfer    Work interaction")
print("1-2(adiabatic)           ",Q12,"KJ        ",round(W12,2))
print("2-3(isothermal)          ",Q23,"KJ        ",W23)
print("3-4(adiabatic)           ",Q34,"KJ        ",round(W34,2))
print("4-1(isothermal)          ",Q41,"KJ        ",W41)
print("Maximum temperature of the cycle: ",round(T2,2),"KJ")
print("Minimum temperature of the cycle: ",round(T1,2),"KJ")
print("Volume at the end of the expansion:",round(v3,4),"m**3")

Process                    Heat transfer    Work interaction
1-2(adiabatic)            0 KJ         -105.51
2-3(isothermal)           40 KJ         40
3-4(adiabatic)            0 KJ         105.51
4-1(isothermal)           -40 KJ         -40
Maximum temperature of the cycle:  585.37 KJ
Minimum temperature of the cycle:  292.68 KJ
Volume at the end of the expansion: 0.1932 m**3


In [ ]:
import math

n = 1
R = 0.082
Cp = 1.008 #Specific heat at const pressure(in kJ/kg):
Cv = 0.721 #Specific heat at const volume(in kJ/kg):
p0 = 7*10**5 #Initial pressure(in Pa):
v0 = 0.12 #Initial volume(in m**3):
T0 = 373
Q01 = n*R*T0*math.log(v1/v0)
Q12 = 0
Q23 = n*R*T2*math.log(v3/v2)
Q30 = 0

T0 = p0*v0/n*R*T0   #Maximum temperature of the cycle(in K):
T1 = T0/2  #Minimum temperature(in K):
W01 = -Q01
W12 = 

#Results:
print("Process                    Heat transfer    Work interaction")
print("1-2(adiabatic)           ",Q12,"KJ        ",round(W12,2))
print("2-3(isothermal)          ",Q23,"KJ        ",W23)
print("3-4(adiabatic)           ",Q34,"KJ        ",round(W34,2))
print("4-1(isothermal)          ",Q41,"KJ        ",W41)
print("Maximum temperature of the cycle: ",round(T2,2),"KJ")
print("Minimum temperature of the cycle: ",round(T1,2),"KJ")
print("Volume at the end of the expansion:",round(v3,4),"m**3")